## PBL: Xây dựng Mô hình Nhận diện Hành vi Bất thường (UCF-Crime)

Đây là Notebook huấn luyện mô hình hạt nhân (core model) cho dự án.
Chúng ta sẽ thực hiện các bước sau:
1.  **Imports:** Tải các thư viện cần thiết.
2.  **Model Definition:** Định nghĩa kiến trúc `CnnRnn` (ResNet50 + LSTM).
3.  **Dataset Definition:** Định nghĩa class `VideoDataset` để tải dữ liệu.
4.  **Hyperparameters:** Thiết lập các tham số huấn luyện.
5.  **Data Loading:** Tạo `train_loader` và `test_loader`.
6.  **Training:** Khởi tạo mô hình và chạy vòng lặp huấn luyện.
7.  **Save Model:** Lưu lại mô hình đã huấn luyện.

### ## 1. Imports Thư viện

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms

import cv2
import os
import numpy as np
import glob
import time
from PIL import Image

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.9.0+cpu
CUDA available: False


### ## 2. Định nghĩa Kiến trúc Mô hình (CnnRnn)

Đây là mô hình hạt nhân kết hợp CNN (để trích xuất đặc trưng không gian từ frame) và RNN/LSTM (để học mối quan hệ thời gian).

In [29]:
class CnnRnn(nn.Module):
    def __init__(self, num_classes, hidden_size=256, num_layers=2, dropout=0.5):
        """
        Args:
            num_classes (int): Số lượng lớp hành vi cần phân loại.
            hidden_size (int): Kích thước của trạng thái ẩn trong LSTM.
            num_layers (int): Số lớp LSTM chồng lên nhau.
            dropout (float): Tỷ lệ dropout.
        """
        super(CnnRnn, self).__init__()

        # --- Phần CNN (ResNet50) ---
        resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        
        # Loại bỏ lớp phân loại cuối cùng (fully connected layer) của ResNet
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        
        # Đóng băng trọng số của ResNet để không huấn luyện lại
        for param in self.resnet.parameters():
            param.requires_grad = False

        # --- Phần RNN (LSTM) ---
        self.lstm = nn.LSTM(
            input_size=resnet.fc.in_features, # Đầu vào là feature size từ ResNet (2048)
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True, # Đầu vào có dạng (batch_size, sequence_length, features)
            dropout=dropout
        )

        # --- Lớp phân loại cuối cùng ---
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, x):
        """
        Args:
            x (Tensor): Tensor đầu vào có kích thước (batch_size, sequence_length, C, H, W)
        """
        batch_size, seq_length, c, h, w = x.shape
        
        # Reshape từ (batch, seq, C, H, W) -> (batch * seq, C, H, W)
        cnn_in = x.view(batch_size * seq_length, c, h, w)
        
        # Đưa từng frame qua CNN
        with torch.no_grad(): # Tắt gradient cho phần resnet đã đóng băng
            cnn_out = self.resnet(cnn_in)
        
        # Reshape lại để đưa vào LSTM
        # Kích thước (batch * seq, 2048, 1, 1) -> (batch, seq, 2048)
        cnn_out = cnn_out.view(batch_size, seq_length, -1)
        
        # Đưa chuỗi features qua LSTM
        lstm_out, _ = self.lstm(cnn_out)
        
        # Ta chỉ cần output của bước thời gian cuối cùng để phân loại
        last_hidden_state = lstm_out[:, -1, :]
        
        # Đưa qua lớp dropout và lớp phân loại
        out = self.dropout_layer(last_hidden_state)
        out = self.fc(out)
        
        return out

### ## 3. Định nghĩa `VideoDataset`

Class này sẽ đọc các video clip từ thư mục `data_clips/train` và `data_clips/test`, trích xuất 1 số lượng frame cố định, và áp dụng các phép biến đổi (transform).

In [30]:
class VideoDataset(Dataset):
    def __init__(self, data_dir, sequence_length=20, transform=None):
        """
        Args:
            data_dir (string): Đường dẫn đến thư mục (train hoặc test).
            sequence_length (int): Số lượng frame cần lấy từ mỗi video.
            transform (callable, optional): Transform áp dụng cho mỗi frame.
        """
        self.data_dir = data_dir
        self.sequence_length = sequence_length
        self.transform = transform
        
        # Tìm tất cả các thư mục con (chính là tên lớp)
        self.classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        
        self.video_paths = [] # Danh sách (đường dẫn video, nhãn (dạng số))
        
        # Quét qua các thư mục con
        for cls_name in self.classes:
            class_dir = os.path.join(data_dir, cls_name)
            class_idx = self.class_to_idx[cls_name]
            
            for video_file in os.listdir(class_dir):
                if video_file.endswith(('.mp4', '.avi', '.x264')):
                    self.video_paths.append((os.path.join(class_dir, video_file), class_idx))
                
        print(f"Đã tìm thấy {len(self.video_paths)} video trong '{data_dir}'.")
        print(f"Có {len(self.classes)} lớp: {self.classes}")

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path, label = self.video_paths[idx]
        
        frames = []
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Lỗi: Không thể mở video {video_path}")
            # Trả về tensor rỗng nếu lỗi
            return torch.zeros(self.sequence_length, 3, 224, 224), -1 

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Chọn ra 'sequence_length' frames rải đều
        frame_indices = np.linspace(0, total_frames - 1, self.sequence_length, dtype=int)
        
        for i in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if ret:
                # OpenCV đọc ảnh dạng BGR, chuyển sang RGB cho PyTorch
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # Chuyển từ mảng numpy sang PIL Image để transform
                pil_img = Image.fromarray(frame)
                
                if self.transform:
                    frame_tensor = self.transform(pil_img)
                frames.append(frame_tensor)
            else:
                # Nếu không đọc được frame (video hỏng/ngắn), lặp lại frame cuối
                if len(frames) > 0:
                    frames.append(frames[-1]) 
                else:
                    # Nếu ngay frame đầu đã lỗi, tạo frame đen
                    frames.append(torch.zeros(3, 224, 224)) # (C, H, W)

        cap.release()
        
        # Stack các frames lại thành (sequence_length, C, H, W)
        video_tensor = torch.stack(frames)
        
        return video_tensor, label

### ## 4. Thiết lập Hyperparameters và Đường dẫn

In [31]:
# --- Tham số ---
SEQUENCE_LENGTH = 20  # Số frame trích xuất từ mỗi video
BATCH_SIZE = 16       # Số video trong 1 batch
NUM_EPOCHS = 25       # Số lượt huấn luyện
LEARNING_RATE = 0.0001 # Tốc độ học

# --- Đường dẫn ---
TRAIN_DIR = 'DATA/data_clips/train'
TEST_DIR = 'DATA/data_clips/test'

# --- Thiết bị ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

Sử dụng thiết bị: cpu


### ## 5. Tạo `Dataset` và `DataLoader`

Chúng ta sẽ định nghĩa các phép biến đổi (resize, normalize) và tạo các loader.

In [32]:
# Định nghĩa các phép biến đổi cho frame
# Phải giống với cách ImageNet được huấn luyện
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

print("Đang tải Train dataset...")
train_dataset = VideoDataset(
    data_dir=TRAIN_DIR, 
    sequence_length=SEQUENCE_LENGTH, 
    transform=data_transform
)

print("\nĐang tải Test dataset...")
test_dataset = VideoDataset(
    data_dir=TEST_DIR, 
    sequence_length=SEQUENCE_LENGTH, 
    transform=data_transform
)

# Tạo DataLoader
# num_workers > 0 để tăng tốc tải dữ liệu (dùng đa luồng)
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=0
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=0
)

# Lấy số lượng lớp từ dataset
NUM_CLASSES = len(train_dataset.classes)
print(f"\nSố lượng lớp phát hiện: {NUM_CLASSES}")

Đang tải Train dataset...
Đã tìm thấy 532 video trong 'DATA/data_clips/train'.
Có 14 lớp: ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'Normal', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']

Đang tải Test dataset...
Đã tìm thấy 4509 video trong 'DATA/data_clips/test'.
Có 14 lớp: ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'Normal', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']

Số lượng lớp phát hiện: 14


### ## 6. Khởi tạo Mô hình, Loss, và Optimizer

In [35]:
# Khởi tạo mô hình
model = CnnRnn(num_classes=NUM_CLASSES).to(device)

# Định nghĩa hàm mất mát (Loss Function)
# CrossEntropyLoss phù hợp cho bài toán phân loại đa lớp
criterion = nn.CrossEntropyLoss()

# Định nghĩa thuật toán tối ưu (Optimizer)
# Adam là một lựa chọn phổ biến và hiệu quả
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# LOGIC TẢI CHECKPOINT
CHECKPOINT_PATH = 'pbl_checkpoint.pth'
start_epoch = 0 # Mặc định bắt đầu từ epoch 0

if os.path.exists(CHECKPOINT_PATH):
    print(f"Phát hiện checkpoint! Đang tải từ '{CHECKPOINT_PATH}'...")
    try:
        checkpoint = torch.load(CHECKPOINT_PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1 # Bắt đầu từ epoch TIẾP THEO
        print(f"Đã tải thành công. Sẽ tiếp tục từ Epoch {start_epoch}")
    except Exception as e:
        print(f"Lỗi khi tải checkpoint: {e}. Bắt đầu huấn luyện lại từ đầu.")
        start_epoch = 0
else:
    print("Không tìm thấy checkpoint. Bắt đầu huấn luyện từ đầu.")

print("\nThông tin mô hình:")
print(model)

Không tìm thấy checkpoint. Bắt đầu huấn luyện từ đầu.

Thông tin mô hình:
CnnRnn(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(in

### ## 7. Vòng lặp Huấn luyện (Training Loop)

Đây là phần chính, nơi mô hình học từ dữ liệu.
Chúng ta sẽ lặp qua `NUM_EPOCHS` lượt:
- Mỗi lượt (epoch) sẽ duyệt qua toàn bộ `train_loader` để cập nhật trọng số.
- Sau mỗi epoch, ta sẽ duyệt qua `test_loader` để đánh giá độ chính xác.

In [34]:
# %% [code]
print("Bắt đầu huấn luyện...")
start_time = time.time()

history = {
    'train_loss': [],
    'test_acc': []
}

# --- (SỬA ĐỔI) Thay 'range(NUM_EPOCHS)' thành 'range(start_epoch, NUM_EPOCHS)' ---
for epoch in range(start_epoch, NUM_EPOCHS):
    epoch_start_time = time.time()
    
    # --- PHA HUẤN LUYỆN (GIỮ NGUYÊN) ---
    model.train()
    running_loss = 0.0
    
    for i, (videos, labels) in enumerate(train_loader):
        videos = videos.to(device)
        labels = labels.to(device)
        
        outputs = model(videos)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if (i + 1) % 20 == 0: 
            print(f'  Epoch [{epoch+1}/{NUM_EPOCHS}], Batch [{i+1}/{len(train_loader)}], Train Loss: {loss.item():.4f}')

    avg_train_loss = running_loss / len(train_loader)
    history['train_loss'].append(avg_train_loss)
    
    # --- PHA ĐÁNH GIÁ (GIỮ NGUYÊN) ---
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for videos, labels in test_loader:
            videos = videos.to(device)
            labels = labels.to(device)
            outputs = model(videos)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    history['test_acc'].append(accuracy)
    
    epoch_time = time.time() - epoch_start_time
    
    # In kết quả của Epoch (GIỮ NGUYÊN)
    print(f'*** === Epoch [{epoch+1}/{NUM_EPOCHS}] === ***')
    print(f'  Thời gian: {epoch_time:.2f}s')
    print(f'  Training Loss: {avg_train_loss:.4f}')
    print(f'  Test Accuracy: {accuracy:.2f} %')
    
    # --- (THÊM MỚI) LƯU CHECKPOINT SAU MỖI EPOCH ---
    print(f"  Đang lưu checkpoint cho Epoch {epoch}...")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_train_loss,
    }, CHECKPOINT_PATH)
    print(f"  Đã lưu checkpoint vào '{CHECKPOINT_PATH}'")
    # ------------------------------------------------
    
    print(f'************************************\n')

total_time = time.time() - start_time
print(f"Hoàn thành huấn luyện! Tổng thời gian: {total_time // 60:.0f} phút {total_time % 60:.0f} giây")

Bắt đầu huấn luyện...


KeyboardInterrupt: 

### ## 8. Lưu Mô hình (Save Model)

Lưu lại trọng số (weights) của mô hình đã huấn luyện để có thể tái sử dụng sau này.

In [ ]:
MODEL_SAVE_PATH = 'pbl6_core_model.pth'
torch.save(model.state_dict(), MODEL_SAVE_PATH)
print(f'Đã lưu mô hình đã huấn luyện vào: {MODEL_SAVE_PATH}')

### ## (Tùy chọn) 9. Vẽ biểu đồ Huấn luyện

Bạn có thể dùng `matplotlib` để xem loss đã giảm và accuracy đã tăng như thế nào.

In [ ]:
import matplotlib.pyplot as plt

# Cần cài đặt: pip install matplotlib
plt.figure(figsize=(12, 5))

# Vẽ Training Loss
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Training Loss')
plt.title('Training Loss qua các Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Vẽ Test Accuracy
plt.subplot(1, 2, 2)
plt.plot(history['test_acc'], label='Test Accuracy', color='orange')
plt.title('Test Accuracy qua các Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()